# DQN implementation with PyTorch using PongNoFrameskip-v4 benchmark.

In this notebook, we implement Deep Q-Network (DQN), one of the rainforcement learning algorithm, using `PyTorch`.  
This code refers to [jmichaux/dqn-pytorch](https://github.com/jmichaux/dqn-pytorch).

In this code, we propose the new method of improve the escape from local-minimum of rainforcement learning. Our method takeing multi-agent approach. In addition, we also define the agent durabiity inspired by Evolutionary computation.

## Setup

In [1]:
!apt-get install -y cmake zlib1g-dev libjpeg-dev xvfb ffmpeg xorg-dev python-opengl libboost-all-dev libsdl2-dev swig freeglut3-dev
!pip install -U gym imageio PILLOW pyvirtualdisplay 'gym[atari]' 'pyglet==1.3.2' pyopengl scipy JSAnimation opencv-python pillow h5py pyyaml hyperdash pyvirtualdisplay hyperdash
!apt-get install xvfb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libjpeg-dev is already the newest version (8c-2ubuntu8).
libjpeg-dev set to manually installed.
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
freeglut3-dev is already the newest version (2.8.1-3).
freeglut3-dev set to manually installed.
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following additional packages will be installed:
  gir1.2-ibus-1.0 libcapnp-0.6.1 libdbus-1-dev libdmx-dev libdmx1
  libfontenc-dev libfs-dev libfs6 libibus-1.0-5 libibus-1.0-dev
  libmirclient-dev libmirclient9 libmircommon-dev libmircommon7
  libmircookie-dev libmircookie2 libmircore-dev libmircore1 libmirprotobuf3
  libpciaccess-dev libpixman-1-dev libprotobuf-dev libprotobuf-lite10
  libpulse-dev libpu

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 8 not upgraded.
Need to get 0 B/783 kB of archives.
After this operation, 2,266 kB of additional disk space will be used.
Selecting previously unselected package xvfb.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.3_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.3) ...
Setting up xvfb (2:1.19.6-1ubuntu4.3) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!cp /content/drive/My\ Drive/Colab\ Notebooks/MT/Utils/xdpyinfo /usr/bin/
!cp /content/drive/My\ Drive/Colab\ Notebooks/MT/Utils/libXxf86dga.* /usr/lib/x86_64-linux-gnu/
!chmod +x /usr/bin/xdpyinfo

In [0]:
!hyperdash signup --github

## Package Import

In [0]:
import copy
from collections import namedtuple
from itertools import count
import math
import random
import numpy as np
import os
import time
import json

import gym
from collections import deque
from hyperdash import Experiment
import cv2

import pyvirtualdisplay
import base64
import IPython

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

## Hyper parameters

In [0]:
# Runtime settings
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")    
Transition = namedtuple('Transion', ('state', 'action', 'next_state', 'reward'))
cv2.ocl.setUseOpenCL(False)
time_stamp = str(int(time.time()))
random.seed(0)
np.random.seed(0)

# Hyper parameters
BATCH_SIZE = 32 # @param
GAMMA = 0.99 # @param
EPS_START = 1 # @param
EPS_END = 0.02 # @param
EPS_DECAY = 1000000 # @param
TARGET_UPDATE = 1000 # @param
DEFAULT_DURABILITY = 1000 # @param
LEARNING_RATE = 1e-4 # @param
INITIAL_MEMORY = 10000 # @param
MEMORY_SIZE = 10 * INITIAL_MEMORY # @param
DEFAULT_DURABILITY_DECREASED_LEVEL = 1 # @param
DEFAULT_DURABILITY_INCREASED_LEVEL = 1 # @param
DURABILITY_CHECK_FREQUENCY = 80 # @param

# Some settings
ENV_NAME = "PongNoFrameskip-v4" # @param
EXP_NAME = "PongNoFrameskip-v4_" + time_stamp # @param
RENDER = False # @param

RUN_NAME = "videos_proposal" # @param
output_directory = os.path.abspath(
    os.path.join(os.path.curdir, "/content/drive/My Drive/Colab Notebooks/MT/Runs", ENV_NAME + "_" + RUN_NAME + "_" + time_stamp))

TRAIN_LOG_FILE_PATH = output_directory + "/" + ENV_NAME + "_train_" + time_stamp + ".log" # @param
TEST_LOG_FILE_PATH = output_directory + "/" + ENV_NAME + "_test_" + time_stamp + ".log" # @param
PARAMETER_LOG_FILE_PATH = output_directory + "/" + ENV_NAME + "_params_" + time_stamp + ".json" # @param

if not os.path.exists(output_directory):
    os.makedirs(output_directory)

hyper_params = {"BATCH_SIZE": BATCH_SIZE, "GAMMA": GAMMA, "EPS_START": EPS_START,
                "EPS_END": EPS_END, "EPS_DECAY": EPS_DECAY,
                "TARGET_UPDATE": TARGET_UPDATE,
                "DEFAULT_DURABILITY": DEFAULT_DURABILITY,
                "LEARNING_RATE": LEARNING_RATE,
                "INITIAL_MEMORY": INITIAL_MEMORY, "MEMORY_SIZE": MEMORY_SIZE,
                "DEFAULT_DURABILITY_DECREASED_LEVEL": DEFAULT_DURABILITY_DECREASED_LEVEL,
                "DURABILITY_CHECK_FREQUENCY": DURABILITY_CHECK_FREQUENCY,
                "ENV_NAME" : ENV_NAME, "EXP_NAME": EXP_NAME, 
                "TRAIN_LOG_FILE_PATH": TRAIN_LOG_FILE_PATH,
                "TEST_LOG_FILE_PATH": TEST_LOG_FILE_PATH,
                "PARAMETER_LOG_FILE_PATH": PARAMETER_LOG_FILE_PATH,
                "RENDER": str(RENDER)}

## Define the Replay memory

In [0]:
class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0
        
    def push(self, *args):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity
        
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)


class PrioritizedReplay(object):
    def __init__(self, capacity):
        pass

## Define the DQNs

Now we define the two types of DQN. One is simple q-network using 3 layers CNN. On the other one is batch normalaized 4 layers CNN.

In [0]:
SQRT2 = math.sqrt(2.0)

ACT = nn.ReLU


class DQN(torch.jit.ScriptModule):
    def __init__(self, in_channels=4, n_actions=14):
        super(DQN, self).__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=8, stride=4),
            ACT(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            ACT(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            ACT()
        )
        self.fc = nn.Sequential(
            nn.Linear(7 * 7 * 64, 512),
            ACT(),
            nn.Linear(512, n_actions)
        )

    @torch.jit.script_method
    def forward(self, x):
        x = x.float() / 255
        x = self.convs(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)


class DDQN(torch.jit.ScriptModule):
    def __init__(self, in_channels=4, n_actions=14):
        __constants__ = ['n_actions']

        super(DDQN, self).__init__()

        self.n_actions = n_actions

        self.convs = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=8, stride=4),
            ACT(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            ACT(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            ACT()
        )
        self.fc_adv = nn.Sequential(
            nn.Linear(7 * 7 * 64, 512),
            ACT(),
            nn.Linear(512, n_actions)
        )

        self.fc_val = nn.Sequential(
            nn.Linear(7 * 7 * 64, 512),
            ACT(),
            nn.Linear(512, 1)
        )

        def scale_grads_hook(module, grad_out, grad_in):
            """scale gradient by 1/sqrt(2) as in the original paper"""
            grad_out = tuple(map(lambda g: g / SQRT2, grad_out))
            return grad_out

        self.fc_adv.register_backward_hook(scale_grads_hook)
        self.fc_val.register_backward_hook(scale_grads_hook)

    @torch.jit.script_method
    def forward(self, x):
        x = x.float() / 255
        x = self.convs(x)
        x = x.view(x.size(0), -1)

        adv = self.fc_adv(x)
        val = self.fc_val(x)

        return val + adv - adv.mean(1).unsqueeze(1)

    @torch.jit.script_method
    def value(self, x):
        x = x.float() / 255
        x = self.convs(x)
        x = x.view(x.size(0), -1)

        return self.fc_val(x)


class LanderDQN(torch.jit.ScriptModule):
    def __init__(self, n_state, n_actions, nhid=64):
        super(LanderDQN, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(n_state, nhid),
            ACT(),
            nn.Linear(nhid, nhid),
            ACT(),
            nn.Linear(nhid, n_actions)
        )

    @torch.jit.script_method
    def forward(self, x):
        x = self.layers(x)
        return x


class RamDQN(torch.jit.ScriptModule):
    def __init__(self, n_state, n_actions):
        super(RamDQN, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(n_state, 256),
            ACT(),
            nn.Linear(256, 128),
            ACT(),
            nn.Linear(128, 64),
            ACT(),
            nn.Linear(64, n_actions)
        )

    @torch.jit.script_method
    def forward(self, x):
        return self.layers(x)


class DQNbn(nn.Module):
    def __init__(self, in_channels=4, n_actions=14):
        """
        Initialize Deep Q Network
        Args:
            in_channels (int): number of input channels
            n_actions (int): number of outputs
        """
        super(DQNbn, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.fc4 = nn.Linear(7 * 7 * 64, 512)
        self.head = nn.Linear(512, n_actions)
        
    def forward(self, x):
        x = x.float() / 255
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.fc4(x.view(x.size(0), -1)))
        return self.head(x)


# class DQN(nn.Module):
#     def __init__(self, in_channels=4, n_actions=14):
#         """
#         Initialize Deep Q Network
#         Args:
#             in_channels (int): number of input channels
#             n_actions (int): number of outputs
#         """
#         super(DQN, self).__init__()
#         self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=8, stride=4)
#         # self.bn1 = nn.BatchNorm2d(32)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
#         # self.bn2 = nn.BatchNorm2d(64)
#         self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
#         # self.bn3 = nn.BatchNorm2d(64)
#         self.fc4 = nn.Linear(7 * 7 * 64, 512)
#         self.head = nn.Linear(512, n_actions)
        
#     def forward(self, x):
#         x = x.float() / 255
#         x = F.relu(self.conv1(x))
#         x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
#         x = F.relu(self.fc4(x.view(x.size(0), -1)))
#         return self.head(x)

## Define the Agent

In [0]:
class Agent:
    def __init__(self, policy_net, target_net, durability, optimizer, name):
        self.policy_net = policy_net
        self.target_net = target_net
        self.target_net.load_state_dict(policy_net.state_dict())
        self.durability = durability
        self.optimizer = optimizer
        self.name = name
        self.memory = ReplayMemory(MEMORY_SIZE)
        self.steps_done = 0
        self.total_reward = 0.0


    def select_action(self, state):
        sample = random.random()
        eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1. * self.steps_done / EPS_DECAY)
        self.steps_done += 1
        if sample > eps_threshold:
            with torch.no_grad():
              return self.policy_net(state.to('cuda')).max(1)[1].view(1,1)
        else:
            return torch.tensor([[random.randrange(4)]], device=device, dtype=torch.long)

    
    def optimize_model(self):
        if len(self.memory) < BATCH_SIZE:
            return
        transitions = self.memory.sample(BATCH_SIZE)
        """
        zip(*transitions) unzips the transitions into
        Transition(*) creates new named tuple
        batch.state - tuple of all the states (each state is a tensor)
        batch.next_state - tuple of all the next states (each state is a tensor)
        batch.reward - tuple of all the rewards (each reward is a float)
        batch.action - tuple of all the actions (each action is an int)    
        """
        batch = Transition(*zip(*transitions))
        
        actions = tuple((map(lambda a: torch.tensor([[a]], device='cuda'), batch.action))) 
        rewards = tuple((map(lambda r: torch.tensor([r], device='cuda'), batch.reward))) 

        non_final_mask = torch.tensor(
            tuple(map(lambda s: s is not None, batch.next_state)),
            device=device, dtype=torch.bool)
        
        non_final_next_states = torch.cat([s for s in batch.next_state
                                        if s is not None]).to('cuda')
        

        state_batch = torch.cat(batch.state).to('cuda')
        action_batch = torch.cat(actions)
        reward_batch = torch.cat(rewards)
        
        state_action_values = self.policy_net(state_batch).gather(1, action_batch)
        
        next_state_values = torch.zeros(BATCH_SIZE, device=device)
        next_state_values[non_final_mask] = self.target_net(non_final_next_states).max(1)[0].detach()
        expected_state_action_values = (next_state_values * GAMMA) + reward_batch
        
        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
        
        self.optimizer.zero_grad()
        loss.backward()
        for param in self.policy_net.parameters():
            param.grad.data.clamp_(-1, 1)
        self.optimizer.step()


    def get_state(self):
        return self.state

      
    def set_state(self, state):
        self.state = state
        

    def set_env(self, env):
        self.env = env


    def get_env(self):
        return self.env


    def set_action(self, action):
        self.action = action


    def get_action(self):
        return self.action


    def get_durability(self):
        return self.durability


    def get_policy_net(self):
        return self.policy_net

    
    def reduce_durability(self, value):
        self.durability = self.durability - value

    def heal_durability(self, value):
        self.durability = self.durability + value
    
    def set_done_state(self, done):
        self.done = done
    

    def set_total_reward(self, reward):
        self.reward = self.reward + reward
    
    def get_total_reward(self):
        return self.total_reward


    def set_step_retrun_value(self, obs, reward, done, info):
        self.obs = obs
        self.reward = reward
        self.done = done
        self.info = info
        

    def is_done(self):
        return self.done

## Define the Environment

**TODO: Make sure to create environment class**

In [0]:
# def make_env(env, stack_frames=True, episodic_life=True, clip_rewards=False, scale=False):
#     if episodic_life:
#         env = EpisodicLifeEnv(env)
#
#     env = NoopResetEnv(env, noop_max=30)
#     env = MaxAndSkipEnv(env, skip=4)
#     if 'FIRE' in env.unwrapped.get_action_meanings():
#         env = FireResetEnv(env)
#
#     env = WarpFrame(env)
#     if stack_frames:
#         env = FrameStack(env, 4)
#     if clip_rewards:
#         env = ClipRewardEnv(env)
#     return env


def get_state(obs):
    state = np.array(obs)
    state = state.transpose((2, 0, 1))
    state = torch.from_numpy(state)
    return state.unsqueeze(0)


class Environment:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.env = self.make_env(self.env)
        
    def get_env(self):
        return self.env


    def make_env(self, env, stack_frames=True, episodic_life=True, clip_rewards=False, scale=False):
        if episodic_life:
            env = EpisodicLifeEnv(env)

        env = NoopResetEnv(self.env, noop_max=30)
        env = MaxAndSkipEnv(self.env, skip=4)
        if 'FIRE' in env.unwrapped.get_action_meanings():
            env = FireResetEnv(self.env)

        env = WarpFrame(env)
        if stack_frames:
            env = FrameStack(env, 4)
        if clip_rewards:
            env = ClipRewardEnv(env)
        return env


    def get_state(obs):
        state = np.array(obs)
        state = state.transpose((2, 0, 1))
        state = torch.from_numpy(state)
        return state.unsqueeze(0)

    
class RewardScaler(gym.RewardWrapper):

    def reward(self, reward):
        return reward * 0.1


class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)


class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=2)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = gym.spaces.Box(low=0, high=255, shape=(shp[0], shp[1], shp[2] * k), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        """Warp frames to 84x84 as done in the Nature paper and later work."""
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = gym.spaces.Box(low=0, high=255,
            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height), interpolation=cv2.INTER_AREA)
        return frame[:, :, None]


class FireResetEnv(gym.Wrapper):
    def __init__(self, env=None):
        """For environments where the user need to press FIRE for the game to start."""
        super(FireResetEnv, self).__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        self.env.reset()
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset()
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset()
        return obs


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env=None):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        super(EpisodicLifeEnv, self).__init__(env)
        self.lives = 0
        self.was_real_done = True
        self.was_real_reset = False

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert somtimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset()
            self.was_real_reset = True
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
            self.was_real_reset = False
        self.lives = self.env.unwrapped.ale.lives()
        return obs


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env=None, skip=4):
        """Return only every `skip`-th frame"""
        super(MaxAndSkipEnv, self).__init__(env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = deque(maxlen=2)
        self._skip = skip

    def step(self, action):
        total_reward = 0.0
        done = None
        for _ in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            self._obs_buffer.append(obs)
            total_reward += reward
            if done:
                break

        max_frame = np.max(np.stack(self._obs_buffer), axis=0)

        return max_frame, total_reward, done, info

    def reset(self):
        """Clear past frame buffer and init. to first obs. from inner env."""
        self._obs_buffer.clear()
        obs = self.env.reset()
        self._obs_buffer.append(obs)
        return obs

class NoopResetEnv(gym.Wrapper):
    def __init__(self, env=None, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        super(NoopResetEnv, self).__init__(env)
        self.noop_max = noop_max
        self.override_num_noops = None
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def step(self, action):
        return self.env.step(action)

    def reset(self):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset()
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = np.random.randint(1, self.noop_max + 1)
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(0)
            if done:
                obs = self.env.reset()
        return obs

## Deprecated code

**Thease code move to agent class.**

@deprecated
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END)* \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            return policy_net(state.to('cuda')).max(1)[1].view(1,1)
    else:
        return torch.tensor([[random.randrange(4)]], device=device, dtype=torch.long)


@deprecated
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    """
    zip(*transitions) unzips the transitions into
    Transition(*) creates new named tuple
    batch.state - tuple of all the states (each state is a tensor)
    batch.next_state - tuple of all the next states (each state is a tensor)
    batch.reward - tuple of all the rewards (each reward is a float)
    batch.action - tuple of all the actions (each action is an int)    
    """
    batch = Transition(*zip(*transitions))
    
    actions = tuple((map(lambda a: torch.tensor([[a]], device='cuda'), batch.action))) 
    rewards = tuple((map(lambda r: torch.tensor([r], device='cuda'), batch.reward))) 

    non_final_mask = torch.tensor(
        tuple(map(lambda s: s is not None, batch.next_state)),
        device=device, dtype=torch.uint8)
    
    non_final_next_states = torch.cat([s for s in batch.next_state
                                       if s is not None]).to('cuda')
    

    state_batch = torch.cat(batch.state).to('cuda')
    action_batch = torch.cat(actions)
    reward_batch = torch.cat(rewards)
    
    state_action_values = policy_net(state_batch).gather(1, action_batch)
    
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))
    
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()


## Degine the train steps

In my research, make this code multi-agent (**Note**: Multi-agent here means multiple independent agents sharing a task environment)

In [0]:
# TODO : To change the deprecated function to Agent clsss fuction
def train(envs, agents, core_env, core_agent, n_episodes, agent_n, exp, render=False):
    """
    Training step.

    In this code, we use the multi-agents to create candidate for core agent.
    The core agent and environment is main RL set. In addition, each agent has
    own environment and durabiliry. Each agent's reward is checked for the
    specified number of episodes, and if an agent is not selected as the
    best-agent, that agent's durability is reduced.

    Parameters
    ----------
    envs: list of Environment
        List of environment for multi-agent
    agents: list of Agent
        List of multi-agents to create candidates for core_agent
    core_env: Environment
        Main environment of this train step
    core_agent: Agent
        Main agent of this train step
    n_episodes: int
        The number of episodes
    agent_n : int
        The number of agent
    exp: Experiment
        The Experiment object used by hyperdash
    render: boolean, default False
        Flag for whether to render the environment
    """
    for episode in range(n_episodes):
        print("episode: {}".format(episode));
        # 0. Initalize the environment, state and agent params
        obs = core_env.reset()
        core_state = get_state(obs)
        core_agent.total_reward = 0.0
        core_agent.set_state(core_state)
        for agent in agents:
            obs = agent.get_env().reset()
            state = get_state(obs)
            agent.set_state(state)
            agent.total_reward = 0.0
            # agent.durability = DEFAULT_DURABILITY


        for t in count():
            #if t % 20 != 0:
            #    print(str(t) + " ", end='')
            #else:
            #    print("\n")
            #    print([agent.get_durability() for agent in agents])
            #    print(str(t) + " ", end='')            """"
            
            # 1. Select action from environment of each agent
            for agent in agents:
                agent.set_env(core_agent.get_env())
                action = agent.select_action(agent.get_state())
                agent.set_action(action)
            
            # 2. Proceed step of each agent
            for agent in agents:
                obs, reward, done, info = agent.get_env().step(agent.get_action())
                agent.set_step_retrun_value(obs, reward, done, info)

                agent.set_total_reward(reward)

                if not done:
                    next_state = get_state(obs)
                else:
                    next_state = None

                reward = torch.tensor([reward], device=device)

                agent.memory.push(agent.get_state(), agent.get_action().to('cpu'),
                                  next_state, reward.to('cpu'))
                agent.set_state(next_state)

                if agent.steps_done > INITIAL_MEMORY:
                    agent.optimize_model()

                    if agent.steps_done % TARGET_UPDATE == 0:
                        agent.target_net.load_state_dict(agent.policy_net.state_dict())
            
            # ---------------
            # Proposal method
            # ---------------
            
            # 3. Select best agent in this step
            reward_list = [agent.get_total_reward() for agent in agents]
            best_agents = [i for i, v in enumerate(reward_list) if v == max(reward_list)]
            best_agent_index = random.choice(best_agents)
            best_agent = agents[best_agent_index]
            best_agent.heal_durability(DEFAULT_DURABILITY_INCREASED_LEVEL)
            
            
            # Best_agent infomation
            # exp.log("Current best agent: {}".format(best_agent.name))

            # 4. Check the agent durability in specified step
            if t % DURABILITY_CHECK_FREQUENCY == 0:
                if len(agents) > 1:
                    index = [i for i in range(len(agents)) if i not in best_agents]
                    for i in index:
                        agents[i].reduce_durability(DEFAULT_DURABILITY_DECREASED_LEVEL)

            # 5. Main step of core agent
            core_agent_action = best_agent.get_action()
            core_agent.set_action(core_agent_action)

            core_obs, core_reward, core_done, core_info = core_agent.get_env().step(
                core_agent.get_action())
            core_agent.set_step_retrun_value(core_obs, core_reward, core_done, core_info)

            core_agent.set_done_state(core_done)
            core_agent.set_total_reward(core_reward)

            if not core_done:
                core_next_state = get_state(core_obs)
            else:
                core_next_state = None

            core_reward = torch.tensor([core_reward], device=device)

            core_agent.memory.push(core_agent.get_state(),
                                   core_agent.get_action().to('cpu'),
                                   core_next_state, core_reward.to('cpu'))
            core_agent.set_state(core_next_state)

            if core_agent.steps_done > INITIAL_MEMORY:
                core_agent.optimize_model()

                if core_agent.steps_done % TARGET_UPDATE == 0:
                    core_agent.target_net.load_state_dict(core_agent.policy_net.state_dict())

            if core_agent.is_done():
                print("\n")
                break
        
        # 6. Swap agent
        if len(agents) > 1 and episode % DURABILITY_CHECK_FREQUENCY == 0:
            for agent, i in zip(agents, range(len(agents))):
                if agent.durability <= 0:
                    del agents[i]

        # ----------------------
        # End of proposal method
        # ----------------------
        
        exp.metric("total_reward", core_agent.get_total_reward())
        out_str = 'Total steps: {} \t Episode: {}/{} \t Total reward: {}'.format(
            core_agent.steps_done, episode, t, core_agent.total_reward)
        if episode % 20 == 0:
            print(out_str)
            out_str = str("\n" + out_str + "\n")
            exp.log(out_str)
        else:
            print(out_str)
            exp.log(out_str)     
        #with open(TRAIN_LOG_FILE_PATH, 'wt') as f:
        #     f.write(out_str)
    env.close()

## Define the test steps

In [0]:
# TODO : To change the deprecated function to Agent clsss fuction
def test(env, n_episodes, policy, exp, render=True):
    # Save video as mp4 on specified directory
    env = gym.wrappers.Monitor(env, './videos/' + 'dqn_pong_video')
    for episode in range(n_episodes):
        obs = env.reset()
        state = env.get_state(obs)
        total_reward = 0.0
        for t in count():
            action = policy(state.to('cuda')).max(1)[1].view(1,1)

            if render:
                env.render()
                time.sleep(0.02)

            obs, reward, done, info = env.step(action)

            total_reward += reward

            if not done:
                next_state = env.get_state(obs)
            else:
                next_state = None

            state = next_state

            if done:
                out_str = "Finished Episode {} with reward {}".format(
                    episode, total_reward)
                print(out_str)
                exp.log(out_str)
                with open(TEST_LOG_FILE_NAME, 'wt') as f:
                    f.write(out_str)
                break

    env.close()

## Main steps

In [0]:
# Create Agent
agents = []

policy_net_0 = DQN(n_actions=4).to(device)
target_net_0 = DQN(n_actions=4).to(device)
optimizer_0 = optim.Adam(policy_net_0.parameters(), lr=LEARNING_RATE)
agents.append(Agent(policy_net_0, target_net_0, DEFAULT_DURABILITY,
                    optimizer_0, "cnn-dqn0"))
agents.append(Agent(policy_net_0, target_net_0, DEFAULT_DURABILITY,
                    optimizer_0, "cnn-dqn1"))
policy_net_1 = DDQN(n_actions=4).to(device)
target_net_1 = DDQN(n_actions=4).to(device)
optimizer_1 = optim.Adam(policy_net_1.parameters(), lr=LEARNING_RATE)
agents.append(Agent(policy_net_1, target_net_1, DEFAULT_DURABILITY,
                    optimizer_1, "cnn-ddqn0"))
agents.append(Agent(policy_net_1, target_net_1, DEFAULT_DURABILITY,
                    optimizer_1, "cnn-ddqn1"))

core_agent = Agent(policy_net_0, target_net_0, DEFAULT_DURABILITY, optimizer_0,
                   "core")

AGENT_N = len(agents)


In [0]:
# time_stamp = str(int(time.time()))
hyper_params["AGENT_N"] = AGENT_N
json_params = json.dumps(hyper_params)
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
with open(PARAMETER_LOG_FILE_PATH, 'wt') as f:
    f.write(json_params)

In [0]:
# Deprecated code
# create networks
# policy_net = DQN(n_actions=4).to(device)
# target_net = DQN(n_actions=4).to(device)
# target_net.load_state_dict(policy_net.state_dict())

In [0]:
# create environment
# TODO: Create Environment class

# env = gym.make(ENV_NAME)
# env = make_env(env)
envs = []
for i in range(AGENT_N):
    env = Environment()
    env = env.get_env()
    envs.append(env)

core_env = Environment()
core_env = core_env.get_env()

for agent, env in zip(agents, envs):
    agent.set_env(env)
core_agent.set_env(core_env)

In [15]:
# setup optimizer
# optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)

# steps_done = 0

# Deprecated
# initialize replay memory
# memory = ReplayMemory(MEMORY_SIZE)

# Hyperdash experiment
exp = Experiment(EXP_NAME, capture_io=False)
print("Learning rate:{}".format(LEARNING_RATE))
exp.param("Learning rate", LEARNING_RATE)
exp.param("Environment", ENV_NAME)
exp.param("Batch size", BATCH_SIZE)
exp.param("Gamma", GAMMA)
exp.param("Episode start", EPS_START)
exp.param("Episode end", EPS_END)
exp.param("Episode decay", EPS_DECAY)
exp.param("Target update", TARGET_UPDATE)
exp.param("Render", str(RENDER))
exp.param("Initial memory", INITIAL_MEMORY)
exp.param("Memory size", MEMORY_SIZE)

Learning rate:0.0001
{ Learning rate: 0.0001 }
{ Environment: PongNoFrameskip-v4 }
{ Batch size: 32 }
{ Gamma: 0.99 }
{ Episode start: 1 }
{ Episode end: 0.02 }
{ Episode decay: 1000000 }
{ Target update: 1000 }
{ Render: False }
{ Initial memory: 10000 }
{ Memory size: 100000 }


100000

In [0]:
# train model
train(envs, agents, core_env, core_agent, 400, AGENT_N, exp)
exp.end()
torch.save(policy_net, output_directory + "/dqn_pong_model")

In [0]:
# EB
exp.end()

In [0]:
# test model
test_env = Environment()
test_env = env.get_env()

policy_net = torch.load(output_directory + "/dqn_pong_model")
exp_test = Experiment(str(EXP_NAME + "_test_step"), capture_io=False)
test(test_env, 1, policy_net, exp_test, render=False)
exp_test.end()

## Video vidualization

In [0]:
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
os.environ["DISPLAY"] = ":" + str(display.display) + "." + str(display.screen)

In [0]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="640" height="480" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return IPython.display.HTML(tag)

In [0]:
embed_mp4("/content/videos/dqn_pong_video/openaigym.video.0.122.video000000.mp4")

In [0]:
# !mv /content/drive/My\ Drive/Colab\ Notebooks/MT/pong_videos /content/drive/My\ Drive/Colab\ Notebooks/MT/pong_videos_1567682751
# !mv /content/dqn_pong_model /content/drive/My\ Drive/Colab\ Notebooks/MT/pong_videos_1567682751/

In [0]:
!mkdir /content/drive/My\ Drive/Colab\ Notebooks/MT/pong_videos_1568005544

In [0]:
!mv ./PongNoFrameskip-v4_*.log /content/drive/My\ Drive/Colab\ Notebooks/MT/pong_videos_1568005544/
!mv ./dqn_pong_model /content/drive/My\ Drive/Colab\ Notebooks/MT/pong_videos_1568005544/
!mv ./videos /content/drive/My\ Drive/Colab\ Notebooks/MT/pong_videos_1568005544/